## Retrieving and preparing the Data


In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from google.colab import drive

keras.utils.set_random_seed(42)

In [17]:
drive.mount('/content/drive', force_remount=True)

train_df = pd.read_csv("/content/drive/MyDrive/GitHub/NLP/lyric_genre_train.csv", index_col=0).astype(str)
test_df = pd.read_csv("/content/drive/MyDrive/GitHub/NLP/lyric_genre_test.csv", index_col=0).astype(str)
val_df = pd.read_csv("/content/drive/MyDrive/GitHub/NLP/lyric_genre_val.csv", index_col=0).astype(str)

print(f"""
Train samples: {train_df.shape[0]}
Validation samples: {val_df.shape[0]}
Test samples: {test_df.shape[0]}
""")

Mounted at /content/drive

Train samples: 48991
Validation samples: 16331
Test samples: 21774



In [18]:
train_df.head()

,Lyric,Genre
0,"Oh, girl. I can't get ready (Can't get ready f...",Pop
1,We met on a rainy evening in the summertime. D...,Pop
2,We carried you in our arms. On Independence Da...,Rock
3,I know he loved you. A long time ago. I ain't ...,Pop
4,Paralysis through analysis. Yellow moral uncle...,Rock


In [19]:
train_df.tail()

,Lyric,Genre
48986,"[Hook]. Beamer, Benz, Or Bentley. Beamer, Benz...",Hip Hop
48987,You never listen to me. I know I'm better off ...,Pop
48988,Things have come to a pretty pass. Our romance...,Pop
48989,"Little baby, on my shoulder. I could fall into...",Pop
48990,Music : Rudolf Schenker. Lyrics: Klaus Meine. ...,Rock


In [20]:
# Let's check the proportion of each label on training

train_df['Genre'].value_counts() / train_df.shape[0]

Rock       0.549448
Pop        0.295136
Hip Hop    0.155416
Name: Genre, dtype: float64

In [21]:
y_train = pd.get_dummies(train_df['Genre']).to_numpy()
y_val = pd.get_dummies(val_df['Genre']).to_numpy()
y_test = pd.get_dummies(test_df['Genre']).to_numpy()

In [22]:
y_train

array([[0, 1, 0],
       [0, 1, 0],
       [0, 0, 1],
       ...,
       [0, 1, 0],
       [0, 1, 0],
       [0, 0, 1]], dtype=uint8)

## Baseline Model (Bag of Words)


In [23]:
max_tokens = 5000
text_vectorization = keras.layers.TextVectorization(
    max_tokens=max_tokens,
    output_mode="multi_hot")


In [25]:
text_vectorization.adapt(train_df['Lyric'])

In [26]:
text_vectorization.get_vocabulary()[-20:]

['eden',
 'dagger',
 'curve',
 'cheddar',
 'brew',
 'appears',
 'vacant',
 'universal',
 'unholy',
 'terrified',
 'stickin',
 'rumble',
 'rug',
 'pam',
 'os',
 'ooohh',
 'motto',
 'marshall',
 'loyalty',
 'legacy']

In [27]:
X_train = text_vectorization(train_df['Lyric'])
X_val = text_vectorization(val_df['Lyric'])
X_test = text_vectorization(test_df['Lyric'])

In [28]:
X_train

<tf.Tensor: shape=(48991, 5000), dtype=float32, numpy=
array([[1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       ...,
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.]], dtype=float32)>

In [29]:
inputs = keras.Input(shape=(max_tokens, ))
x = keras.layers.Dense(8, activation="relu")(inputs)
outputs = keras.layers.Dense(3, activation="softmax")(x)

model = keras.Model(inputs, outputs)

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 5000)]            0         
                                                                 
 dense (Dense)               (None, 8)                 40008     
                                                                 
 dense_1 (Dense)             (None, 3)                 27        
                                                                 
Total params: 40035 (156.39 KB)
Trainable params: 40035 (156.39 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [30]:
model.compile(optimizer="adam",
              loss="categorical_crossentropy",
              metrics=["accuracy"])

In [31]:
# Fit model
model.fit(x=X_train, y=y_train,
          validation_data=(X_val, y_val),
          epochs=10,
          batch_size=32)

Epoch 1/10
1531/1531 [==============================] - 6s 3ms/step - loss: 0.6167 - accuracy: 0.7311 - val_loss: 0.5701 - val_accuracy: 0.7529
Epoch 2/10
1531/1531 [==============================] - 5s 4ms/step - loss: 0.5200 - accuracy: 0.7765 - val_loss: 0.5730 - val_accuracy: 0.7497
Epoch 3/10
1531/1531 [==============================] - 5s 3ms/step - loss: 0.4820 - accuracy: 0.7919 - val_loss: 0.5849 - val_accuracy: 0.7434
Epoch 4/10
1531/1531 [==============================] - 5s 3ms/step - loss: 0.4509 - accuracy: 0.8067 - val_loss: 0.6015 - val_accuracy: 0.7415
Epoch 5/10
1531/1531 [==============================] - 7s 4ms/step - loss: 0.4227 - accuracy: 0.8207 - val_loss: 0.6270 - val_accuracy: 0.7407
Epoch 6/10
1531/1531 [==============================] - 5s 3ms/step - loss: 0.3987 - accuracy: 0.8324 - val_loss: 0.6586 - val_accuracy: 0.7382
Epoch 7/10
1531/1531 [==============================] - 5s 3ms/step - loss: 0.3766 - accuracy: 0.8433 - val_loss: 0.6785 - val_accuracy:

In [32]:
model.evaluate(x=X_test, y=y_test)

681/681 [==============================] - 1s 2ms/step - loss: 0.8323 - accuracy: 0.7203


[0.8323413133621216, 0.7202627062797546]

## Word Embeddings



In [33]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip -q glove.6B.zip

--2024-03-07 05:34:09--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2024-03-07 05:34:09--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2024-03-07 05:34:09--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

In [34]:
embedding_dim = 100
path_to_glove_file = f"glove.6B.{embedding_dim}d.txt"

embeddings_index = {}
with open(path_to_glove_file) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print(f"Found {len(embeddings_index)} word vectors.")

Found 400000 word vectors.


In [35]:
embeddings_index["movie"]

array([ 0.38251  ,  0.14821  ,  0.60601  , -0.51533  ,  0.43992  ,
        0.061053 , -0.62716  , -0.025385 ,  0.1643   , -0.22101  ,
        0.14423  , -0.37213  , -0.21683  , -0.08895  ,  0.097904 ,
        0.6561   ,  0.64455  ,  0.47698  ,  0.83849  ,  1.6486   ,
        0.88922  , -0.1181   , -0.012465 , -0.52082  ,  0.77854  ,
        0.48723  , -0.014991 , -0.14127  , -0.34747  , -0.29595  ,
        0.1028   ,  0.57191  , -0.045594 ,  0.026443 ,  0.53816  ,
        0.32257  ,  0.40788  , -0.043599 , -0.146    , -0.48346  ,
        0.32036  ,  0.55086  , -0.76259  ,  0.43269  ,  0.61753  ,
       -0.36503  , -0.60599  , -0.79615  ,  0.3929   , -0.23668  ,
       -0.34719  , -0.61201  ,  0.54747  ,  0.94812  ,  0.20941  ,
       -2.7771   , -0.6022   ,  0.8495   ,  1.2549   ,  0.017893 ,
       -0.041901 ,  2.1147   , -0.026618 , -0.28104  ,  0.68124  ,
       -0.14165  ,  0.99249  ,  0.49879  , -0.67538  ,  0.6417   ,
        0.42303  , -0.27913  ,  0.063403 ,  0.68909  , -0.3618

In [36]:
embeddings_index["film"]

array([ 0.19916 , -0.049702,  0.24579 , -0.32281 ,  0.89768 , -0.1278  ,
       -0.49506 ,  0.20814 , -0.20046 , -0.20604 ,  0.038292, -0.67277 ,
       -0.12689 , -0.18766 , -0.10277 ,  0.73128 ,  0.82408 ,  0.087288,
        0.69255 ,  1.3107  ,  0.49113 , -0.38097 ,  0.24338 , -0.27813 ,
        0.62506 ,  0.35978 ,  0.42041 , -0.24529 ,  0.14861 , -0.26726 ,
       -0.56262 ,  0.63843 , -0.54153 ,  0.36537 ,  0.20545 , -0.16604 ,
        0.72434 ,  0.29961 , -0.42501 , -0.35932 , -0.089288,  0.48752 ,
       -1.0927  ,  0.88818 ,  0.89941 , -0.7541  , -0.35492 , -0.76396 ,
        0.27468 ,  0.2757  , -0.48152 , -0.41399 ,  0.64489 ,  1.148   ,
       -0.29131 , -2.9387  , -0.83162 ,  0.95586 ,  1.1623  , -0.42502 ,
        0.15486 ,  2.2326  , -0.31339 , -0.030228,  0.79802 , -0.41302 ,
        0.72885 ,  0.7296  , -0.31909 ,  0.8956  ,  0.34625 ,  0.2923  ,
        0.40056 ,  0.78985 , -0.43999 ,  0.24698 , -0.46548 ,  0.055886,
       -0.62603 , -0.036487, -0.65429 ,  0.10563 , 

In [37]:
max_length = 300 #90% of songs
max_tokens = 5000

text_vectorization = keras.layers.TextVectorization(
    max_tokens=max_tokens,
    output_mode="int",
    output_sequence_length=max_length,
)

We haven't seen the `output_sequence_length` argument before. This basically truncates any doc to the length specified by the argument. If the document is shorter it is padded with the empty tokens (i.e. tokens that index to 0)

In [38]:
text_vectorization.adapt(train_df['Lyric'])

In [39]:
X_train = text_vectorization(train_df['Lyric'])
X_val = text_vectorization(val_df['Lyric'])
X_test = text_vectorization(test_df['Lyric'])

In [40]:
X_train

<tf.Tensor: shape=(48991, 300), dtype=int64, numpy=
array([[  40,   83,    4, ...,   22,  729,    3],
       [  20,  649,   13, ...,    0,    0,    0],
       [  20, 2872,    3, ...,    0,    0,    0],
       ...,
       [ 153,   66,   62, ...,    0,    0,    0],
       [ 119,   51,   13, ...,    0,    0,    0],
       [ 358,    1,    1, ...,    0,    0,    0]])>

In [44]:
vocabulary = text_vectorization.get_vocabulary()
word_index = dict(zip(vocabulary, range(len(vocabulary))))

counter = 0
embedding_matrix = np.zeros((max_tokens, embedding_dim))
for word, i in word_index.items():
   if i < max_tokens:
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
     embedding_matrix[i] = embedding_vector
   else: counter += 1

In [45]:
embedding_matrix.shape

(5000, 100)

In [46]:
embedding_layer = keras.layers.Embedding(
    max_tokens,
    embedding_dim,
    embeddings_initializer= keras.initializers.Constant(embedding_matrix),
    trainable=False,
)

In [47]:
print(f'There are {counter} words on our vocabulary not present in the Glove embedding')
print(f'This is roughly {counter / max_tokens * 100:.2f}% of the vocabulary')
print('These words will be represented by a vector of 0 in all entries in the embedding')

There are 0 words on our vocabulary not present in the Glove embedding
This is roughly 0.00% of the vocabulary
These words will be represented by a vector of 0 in all entries in the embedding


In [48]:
inputs = keras.Input(shape=(max_length,))
embedded = embedding_layer(inputs) # 300 x 100 table comes out
embedded = keras.layers.GlobalAveragePooling1D()(embedded) # 100-element vector
x = keras.layers.Dense(8)(embedded)
x = keras.layers.Dropout(0.5)(x)
outputs = keras.layers.Dense(3, activation="softmax")(x)

model = keras.Model(inputs, outputs)

model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 300)]             0         
                                                                 
 embedding (Embedding)       (None, 300, 100)          500000    
                                                                 
 global_average_pooling1d (  (None, 100)               0         
 GlobalAveragePooling1D)                                         
                                                                 
 dense_2 (Dense)             (None, 8)                 808       
                                                                 
 dropout (Dropout)           (None, 8)                 0         
                                                                 
 dense_3 (Dense)             (None, 3)                 27        
                                                           

In [49]:
model.compile(optimizer="adam",
              loss="categorical_crossentropy",
              metrics=["accuracy"])

In [50]:
# Fit model
model.fit(x=X_train, y=y_train,
          validation_data=(X_val, y_val),
          epochs=10,
          batch_size=32,)

Epoch 1/10
1531/1531 [==============================] - 7s 4ms/step - loss: 0.8919 - accuracy: 0.5920 - val_loss: 0.7926 - val_accuracy: 0.6449
Epoch 2/10
1531/1531 [==============================] - 7s 5ms/step - loss: 0.8093 - accuracy: 0.6355 - val_loss: 0.7671 - val_accuracy: 0.6636
Epoch 3/10
1531/1531 [==============================] - 11s 7ms/step - loss: 0.7955 - accuracy: 0.6432 - val_loss: 0.7536 - val_accuracy: 0.6742
Epoch 4/10
1531/1531 [==============================] - 8s 5ms/step - loss: 0.7939 - accuracy: 0.6476 - val_loss: 0.7494 - val_accuracy: 0.6768
Epoch 5/10
1531/1531 [==============================] - 7s 5ms/step - loss: 0.7875 - accuracy: 0.6487 - val_loss: 0.7523 - val_accuracy: 0.6670
Epoch 6/10
1531/1531 [==============================] - 8s 5ms/step - loss: 0.7859 - accuracy: 0.6520 - val_loss: 0.7440 - val_accuracy: 0.6758
Epoch 7/10
1531/1531 [==============================] - 10s 7ms/step - loss: 0.7832 - accuracy: 0.6533 - val_loss: 0.7433 - val_accurac

In [51]:
embedding_layer = keras.layers.Embedding(
    max_tokens,
    embedding_dim,
    embeddings_initializer=keras.initializers.Constant(embedding_matrix),
    trainable=True,
)

inputs = keras.Input(shape=(max_length,))
embedded = embedding_layer(inputs)
embedded = keras.layers.GlobalAveragePooling1D()(embedded)
x = keras.layers.Dense(8)(embedded)
x = keras.layers.Dropout(0.5)(x)
outputs = keras.layers.Dense(3, activation="softmax")(x)

model = keras.Model(inputs, outputs)
model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 300)]             0         
                                                                 
 embedding_1 (Embedding)     (None, 300, 100)          500000    
                                                                 
 global_average_pooling1d_1  (None, 100)               0         
  (GlobalAveragePooling1D)                                       
                                                                 
 dense_4 (Dense)             (None, 8)                 808       
                                                                 
 dropout_1 (Dropout)         (None, 8)                 0         
                                                                 
 dense_5 (Dense)             (None, 3)                 27        
                                                           

In [52]:
model.compile(optimizer="adam",
              loss="categorical_crossentropy",
              metrics=["accuracy"])


In [53]:
# Fit model
model.fit(x=X_train, y=y_train,
          validation_data=(X_val, y_val),
          epochs=10,
          batch_size=32,)

Epoch 1/10
1531/1531 [==============================] - 24s 15ms/step - loss: 0.7847 - accuracy: 0.6632 - val_loss: 0.6572 - val_accuracy: 0.7216
Epoch 2/10
1531/1531 [==============================] - 20s 13ms/step - loss: 0.6763 - accuracy: 0.7114 - val_loss: 0.6253 - val_accuracy: 0.7305
Epoch 3/10
1531/1531 [==============================] - 21s 14ms/step - loss: 0.6445 - accuracy: 0.7255 - val_loss: 0.6119 - val_accuracy: 0.7371
Epoch 4/10
1531/1531 [==============================] - 24s 15ms/step - loss: 0.6275 - accuracy: 0.7344 - val_loss: 0.6126 - val_accuracy: 0.7386
Epoch 5/10
1531/1531 [==============================] - 18s 11ms/step - loss: 0.6132 - accuracy: 0.7398 - val_loss: 0.6184 - val_accuracy: 0.7317
Epoch 6/10
1531/1531 [==============================] - 21s 14ms/step - loss: 0.6016 - accuracy: 0.7457 - val_loss: 0.6119 - val_accuracy: 0.7370
Epoch 7/10
1531/1531 [==============================] - 21s 14ms/step - loss: 0.5893 - accuracy: 0.7482 - val_loss: 0.6178 -

In [54]:
model.evaluate(x=X_test, y=y_test)

681/681 [==============================] - 2s 2ms/step - loss: 0.6440 - accuracy: 0.7328


[0.6440224051475525, 0.7328006029129028]

In [55]:
def lyric_predict(phrase):
    raw_text_data = tf.convert_to_tensor([[phrase],])

    vect_data = text_vectorization(raw_text_data)
    predictions = model.predict(vect_data)
    predictions
    print(f"{float(predictions[0,0] * 100):.2f} % Hip-Hop")
    print(f"{float(predictions[0,1] * 100):.2f} % Pop")
    print(f"{float(predictions[0,2] * 100):.2f} % Rock")

In [56]:
phrase = '''I grew up on the crime side, the New York Times side
Stayin' alive was no jive
Had secondhands, Mom's bounced on old man
So then we moved to Shaolin land
A young youth, yo, rockin' the gold tooth, 'Lo goose
Only way I begin the G off was drug loot
And let's start it like this, son
Rollin' with this one and that one, pullin' out gats for fun
But it was just a dream for the teen
Who was a fiend, started smokin' woolies at 16
And runnin' up in gates and doin' hits for high stakes
Makin' my way on fire escapes
No question, I would speed for cracks and weed
The combination made my eyes bleed
No question, I would flow off and try to get the dough all
Stickin' up white boys in ball courts
My life got no better, same damn 'Lo sweater
Times is rough and tough like leather
Figured out I went the wrong route
So I got with a sick-ass clique and went all out
Catchin' keys from 'cross seas
Rollin' in MPV's, every week we made forty G's
Yo, ****, respect mine, or here go the TEC-9
Ch-chick-pow! Move from the gate now'''

lyric_predict(phrase)

1/1 [==============================] - 0s 142ms/step
63.13 % Hip-Hop
14.28 % Pop
22.59 % Rock


In [57]:
phrase = '''I walked through the door with you
The air was cold
But something about it felt like home somehow
And I, left my scarf there at your sister's house
And you've still got it in your drawer even now
Oh, your sweet disposition
And my wide-eyed gaze
We're singing in the car, getting lost upstate
Autumn leaves falling down like pieces into place
And I can picture it after all these days
And I know it's long gone and that magic's not here no more
And I might be okay but I'm not fine at all
'Cause there we are again on that little town street
You almost ran the red 'cause you were lookin' over at me
Wind in my hair, I was there
I remember it all too well
Photo album on the counter
Your cheeks were turning red
You used to be a little kid with glasses in a twin-sized bed
And your mother's telling stories 'bout you on the tee-ball team
You told me 'bout your past thinking your future was me
And I know it's long gone and there was nothing else I could do
And I forget about you long enough to forget why I needed to
'Cause there we are again in the middle of the night
We're dancing 'round the kitchen in the refrigerator light
Down the stairs, I was there
I remember it all too well, yeah
And maybe we got lost in translation
Maybe I asked for too much
But maybe this thing was a masterpiece 'til you tore it all up
Running scared, I was there
I remember it all too well
And you call me up again just to break me like a promise
So casually cruel in the name of being honest
I'm a crumpled up piece of paper lying here
'Cause I remember it all, all, all
Too well'''

lyric_predict(phrase)

1/1 [==============================] - 0s 31ms/step
3.82 % Hip-Hop
49.45 % Pop
46.73 % Rock


## Adding Context with Bigrams


In [58]:
# Text Vectorization layer using bigrams
text_vectorization = keras.layers.TextVectorization(
    ngrams=2,
    output_mode="multi_hot")

In [59]:
text_vectorization.adapt(["the cat sat on a mat."])

In [60]:
text_vectorization.get_vocabulary()

['[UNK]',
 'the cat',
 'the',
 'sat on',
 'sat',
 'on a',
 'on',
 'mat',
 'cat sat',
 'cat',
 'a mat',
 'a']